In [27]:
# Here we compute the ROUGE-1 and ROUGE-2
# Here, we have to first process the 3 sentences. Not adjacent -- form unigrams and bigrams -- compute ROUGE-N precision, ROUGE-N recall
# ROUGE-N  f1 score
# Also compute the ROUGE-L score

# Run this code using Python 2.7, otherwise the string.translate() does not work

from PyRouge.pyrouge import Rouge
import pandas as pd
from nltk import word_tokenize
from nltk.util import ngrams
import string
import numpy as np
import math
from scipy import stats

In [ ]:
r_summ_evaluate = Rouge()

In [38]:
# Computing ROUGE-L precision, recall and F1 score



# Trying with a sample summary
# system_generated_summary = " The Kyrgyz President pushed through the law requiring the use of ink during the upcoming Parliamentary and Presidential elections In an effort to live up to its reputation in the 1990s as an island of democracy. The use of ink is one part of a general effort to show commitment towards more open elections. improper use of this type of ink can cause additional problems as the elections in Afghanistan showed. The use of ink and readers by itself is not a panacea for election ills."
# manual_summmary = " The use of invisible ink and ultraviolet readers in the elections of the Kyrgyz Republic which is a small, mountainous state of the former Soviet republic, causing both worries and guarded optimism among different sectors of the population. Though the actual technology behind the ink is not complicated, the presence of ultraviolet light (of the kind used to verify money) causes the ink to glow with a neon yellow light. But, this use of the new technology has caused a lot of problems. "

# print(r_summ_evaluate.rouge_l([system_generated_summary], [manual_summmary]))

# Here we read the file which mentions the 3 sentences for each technique, in three adjacent columns
#summ_data_readpath = '/home/soumyadeep/PycharmProjects/MSThesisWorkMay2019/JournalCodes/data/GoldDataOnly36_12thSept.csv'
# 1 : X.AUTHID, 2 : spec_domain, 3: site.content  4-6: GOLD, 7-9 : LEAD, 10-12 : RAND, 13-15 : INCONST, 16-18 : INCONST_NEG, 19-21 : HYBRID1

#summ_data_readpath = '/home/soumyadeep/PycharmProjects/MSThesisWorkMay2019/JournalCodes/data/GoldDataAllAspectsWithHybrid_13thSept.csv'
# 1 : X.AUTHID, 2 : spec_domain, 3: site.content  4-6: GOLD, 7-9: LEAD, 10-12: Hybrid, 13-15: Incons, 16-18: Neg, 19-21: Ctr, 22-24: InconsNeg
summ_data_readpath = 'D:/PycharmProjects/MSThesisWorkMay2019/JournalCodes/data/GoldDataAllAspects_Annotate_Partial_17thSept.csv'

summ_data = pd.read_csv(summ_data_readpath)
summ_data = summ_data.fillna("")

rand_data_readpath = 'D:/Documents/MSThesisWork/AdobeBrandConsistency/genData1/GoldDataWithINFO_16thJune.csv'
rand_data = pd.read_csv(rand_data_readpath)
rand_data = rand_data.fillna("")



In [4]:
summ_data.head()

,X.AUTHID,spec_domain,site.content,GOLD1,GOLD2,GOLD3,LEAD1,LEAD2,LEAD3,Hybrid1,...,Neg3,Ctr1,Ctr2,Ctr3,InconsNeg1,InconsNeg2,InconsNeg3,LeadN1,LeadN2,LeadN3
0,b'http://www.mastec.com/press-release/1799/1/M...,mastec,"b'August 24, 2011;CORAL GABLES, Fla., Aug. 24,...","August 24, 2011;CORAL GABLES, Fla., Aug. 24, 2...","As of ;, MasTec had appro ; in outstanding loa...","MasTec\'s Chief E Officer, added, """"We are ver...","August 24, 2011;CORAL GABLES, Fla., Aug. 24, 2...",The amendment e the Company\'s e credit facili...,The amended credit facility includes more favo...,"August 24, 2011;CORAL GABLES, Fla., Aug. 24, 2...",...,While our previous credit facility did not e u...,"August 24, 2011;CORAL GABLES, Fla., Aug. 24, 2...","As of ;, MasTec had appro ; in outstanding loa...",While our previous credit facility did not e u...,"As of ;, MasTec had appro ; in outstanding loa...","Based on our current metrics, availability und...","Finally, interest rates and on-going fees, sub...","August 24, 2011;CORAL GABLES, Fla., Aug. 24, 2...",The amendment e the Company\'s e credit facili...,"As of ;, MasTec had appro ; in outstanding loa..."
1,b'http://www.rpminc.com/news-releases/news-rel...,rpminc,"MEDINA, Ohio, April 3 /PRNewswire-FirstCall/ -...","RPM International Inc., a holding company, own...","MEDINA, Ohio, April 3 /PRNewswire-FirstCall/ -...",,"MEDINA, Ohio, April 3 /PRNewswire-FirstCall/ -...",The results will be issued via newswire and wi...,; Management will host a corresponding confere...,"; RPM International Inc., a holding company, o...",...,;RPM International Inc. (NYSE: RPM) owns subsi...,"; RPM International Inc., a holding company, o...",RPM's industrial products include roofing syst...,;RPM International Inc. (NYSE: RPM) owns subsi...,"; RPM International Inc., a holding company, o...",RPM's industrial products include roofing syst...,;RPM International Inc. (NYSE: RPM) owns subsi...,"MEDINA, Ohio, April 3 /PRNewswire-FirstCall/ -...",The call can be accessed by dialing 913-981-5507.,Participants are asked to call the assigned nu...
2,b'http://www.rpminc.com/news-releases/news-rel...,rpminc,"MEDINA, Ohio, Dec. 23 /PRNewswire-FirstCall/ ...","MEDINA, Ohio, Dec. 23 /PRNewswire-FirstCall/ -...",International Inc. (NYSE: RPM) owns subsidiar...,,"MEDINA, Ohio, Dec. 23 /PRNewswire-FirstCall/ ...",The results will be issued via newswire and wi...,A conference call to discuss earnings will beg...,Only the financial analysts will be permitted ...,...,"RPM International Inc., a holding company, own...",Only the financial analysts will be permitted ...,The access code is 392850.,"RPM International Inc., a holding company, own...",Only the financial analysts will be permitted ...,The access code is 392850.,"RPM International Inc., a holding company, own...","MEDINA, Ohio, Dec. 23 /PRNewswire-FirstCall/ ...",The call can be accessed by dialing 913-981-5508.,Participants are asked to call the assigned nu...
3,b'http://www.rpminc.com/news-releases/news-rel...,rpminc,"MEDINA, Ohio, March 22 /PRNewswire-FirstCall/...","The call, which last appro one hour, will be o...",RPM's consumer products are used by profession...,"MEDINA, Ohio, March 22 /PRNewswire-FirstCall/ ...","MEDINA, Ohio, March 22 /PRNewswire-FirstCall/...",The results be issued via newswire and also wi...,; (Logo: http:/www.newscom.com/cgi-bin/prnh/20...,"MEDINA, Ohio, March 22 /PRNewswire-FirstCall/...",...,"CONTACT: Glenn R. Hasman, vice president of fi...",; (Logo: http:/www.newscom.com/cgi-bin/prnh/20...,The media and all participants will be in a li...,"CONTACT: Glenn R. Hasman, vice president of fi...","MEDINA, Ohio, March 22 /PRNewswire-FirstCall/...",; (Logo: http:/www.newscom.com/cgi-bin/prnh/20...,"CONTACT: Glenn R. Hasman, vice president of fi...","MEDINA, Ohio, March 22 /PRNewswire-FirstCall/...",The results be issued via newswire and also wi...,; (Logo: http:/www.newscom.com/cgi-bin/prnh/20...
4,b'http://www.shell.com/media/speeches-and-arti...,shell,b'Shell 

In [5]:
def rouge_one(own_summ, gold_summ):
    # Lets go with unique words only and no preprocessing. But still look into it.
    own_unigrams = set(own_summ)
    gold_unigrams = set(gold_summ)

    ovlp_words = own_unigrams.intersection(gold_unigrams)
    # print(ovlp_words)

    if len(own_unigrams) > 0:
        prec = len(ovlp_words) * 1.0 / len(own_unigrams)
    else:
        prec = 0.0

    if len(gold_unigrams) > 0:
        rec = len(ovlp_words) * 1.0 / len(gold_unigrams)
    else:
        rec = 0.0
    if prec == 0.0 and rec == 0.0:
        fscore = 0.0
    else:
        fscore = (2.0 * prec * rec) / (prec + rec)
    return fscore

In [23]:
article1 = "The sun rises in the east and sets in the west. Today I felt really alive and the sound of the morning train made fall in love with my beloved home ever again."

# Here we evaluate our generated 3 sentences with the Gold standard sentences
# For ROUGE-1
# rouge_one(article1, article1)

# For ROUGE-2
# rouge_N(article1, article1, 2)

# For ROUGE-L
# r_summ_evaluate.rouge_l([article1], [article1])
#rouge1_rand_list = list()
rouge1_lead_list = list()
rouge1_hybrid1_list = list()
rouge1_inconst_list = list()
rouge1_neg_list = list()
rouge1_ctr_list = list()
rouge1_inconst_neg_list = list()

#rouge2_rand_list = list()
rouge2_lead_list = list()
rouge2_hybrid1_list = list()
rouge2_inconst_list = list()
rouge2_neg_list = list()
rouge2_ctr_list = list()
rouge2_inconst_neg_list = list()


#rouge_lcs_rand_list = list()
rouge_lcs_lead_list = list()
rouge_lcs_hybrid1_list = list()
rouge_lcs_inconst_list = list()
rouge_lcs_neg_list = list()
rouge_lcs_ctr_list = list()
rouge_lcs_inconst_neg_list = list()



In [47]:
rouge1_rand_list = list()
rouge2_rand_list = list()
rouge_lcs_rand_list = list()

In [48]:
summ_data.columns

Index(['X.AUTHID', 'spec_domain', 'site.content', 'GOLD1', 'GOLD2', 'GOLD3',
       'LEAD1', 'LEAD2', 'LEAD3', 'Hybrid1', 'Hybrid2', 'Hybrid3', 'Incons1',
       'Incons2', 'Incons3', 'Neg1', 'Neg2', 'Neg3', 'Ctr1', 'Ctr2', 'Ctr3',
       'InconsNeg1', 'InconsNeg2', 'InconsNeg3', 'LeadN1', 'LeadN2', 'LeadN3'],
      dtype='object')

In [49]:
#table = string.maketrans("", "")
# 1 : X.AUTHID, 2 : spec_domain, 3: site.content  4-6: GOLD, 7-9: LEAD, 10-12: Hybrid, 13-15: Incons, 16-18: Neg, 19-21: Ctr, 22-24: InconsNeg
# We have GOLD-3 data for only 36 web articles
for row_ind in range(summ_data.shape[0]):
    gold_list1 = list(summ_data.iloc[row_ind, 3:6])
    gold_list = [x for x in gold_list1 if x == x]
    
        
    gold1_tokens = word_tokenize((summ_data.iloc[row_ind, 3]).translate(str.maketrans('','', string.punctuation)))
    gold2_tokens = word_tokenize((summ_data.iloc[row_ind, 4]).translate(str.maketrans('','', string.punctuation)))
    gold3_tokens = word_tokenize((summ_data.iloc[row_ind, 5]).translate(str.maketrans('','', string.punctuation)))
    
    rand1_tokens = word_tokenize((rand_data.iloc[row_ind, 9]).translate(str.maketrans('','', string.punctuation)))
    rand2_tokens = word_tokenize((rand_data.iloc[row_ind, 10]).translate(str.maketrans('','', string.punctuation)))
    rand3_tokens = word_tokenize((rand_data.iloc[row_ind, 11]).translate(str.maketrans('','', string.punctuation)))

    '''
    lead1_tokens = word_tokenize((summ_data.iloc[row_ind, 6]).translate(str.maketrans('','', string.punctuation)))
    lead2_tokens = word_tokenize((summ_data.iloc[row_ind, 7]).translate(str.maketrans('','', string.punctuation)))
    lead3_tokens = word_tokenize((summ_data.iloc[row_ind, 8]).translate(str.maketrans('','', string.punctuation)))

    hybrid1_tokens = word_tokenize((summ_data.iloc[row_ind, 9]).translate(str.maketrans('','', string.punctuation)))
    hybrid2_tokens = word_tokenize((summ_data.iloc[row_ind, 10]).translate(str.maketrans('','', string.punctuation)))
    hybrid3_tokens = word_tokenize((summ_data.iloc[row_ind, 11]).translate(str.maketrans('','', string.punctuation)))
    
    inconst1_tokens = word_tokenize((summ_data.iloc[row_ind, 12]).translate(str.maketrans('','', string.punctuation)))
    inconst2_tokens = word_tokenize((summ_data.iloc[row_ind, 13]).translate(str.maketrans('','', string.punctuation)))
    inconst3_tokens = word_tokenize((summ_data.iloc[row_ind, 14]).translate(str.maketrans('','', string.punctuation)))

    neg1_tokens = word_tokenize((summ_data.iloc[row_ind, 15]).translate(str.maketrans('','', string.punctuation)))
    neg2_tokens = word_tokenize((summ_data.iloc[row_ind, 16]).translate(str.maketrans('','', string.punctuation)))
    neg3_tokens = word_tokenize((summ_data.iloc[row_ind, 17]).translate(str.maketrans('','', string.punctuation)))

    ctr1_tokens = word_tokenize((summ_data.iloc[row_ind, 18]).translate(str.maketrans('','', string.punctuation)))
    ctr2_tokens = word_tokenize((summ_data.iloc[row_ind, 19]).translate(str.maketrans('','', string.punctuation)))
    ctr3_tokens = word_tokenize((summ_data.iloc[row_ind, 20]).translate(str.maketrans('','', string.punctuation)))

    inconst_neg1_tokens = word_tokenize((summ_data.iloc[row_ind, 21]).translate(str.maketrans('','', string.punctuation)))
    inconst_neg2_tokens = word_tokenize((summ_data.iloc[row_ind, 22]).translate(str.maketrans('','', string.punctuation)))
    inconst_neg3_tokens = word_tokenize((summ_data.iloc[row_ind, 23]).translate(str.maketrans('','', string.punctuation)))
    '''

    if len(gold_list) < 3:
        prec_range_val = 2
        gold_side_1 = gold1_tokens + gold2_tokens
        rand_side_1 = rand1_tokens + rand2_tokens
        '''
        hybrid_side_1 = hybrid1_tokens + hybrid2_tokens
        lead_side_1 = lead1_tokens + lead2_tokens
        inconst_side_1 = inconst1_tokens + inconst2_tokens
        neg_side_1 = neg1_tokens + neg2_tokens
        ctr_side_1 = ctr1_tokens + ctr2_tokens
        inc_neg_side_1 = inconst_neg1_tokens + inconst_neg2_tokens
        '''
        
        gold_side_2 = list(ngrams(gold1_tokens, 2)) + list(ngrams(gold2_tokens, 2))
        rand_side_2 = list(ngrams(rand1_tokens, 2)) + list(ngrams(rand2_tokens, 2))
        '''
        hybrid_side_2 = list(ngrams(hybrid1_tokens, 2)) + list(ngrams(hybrid2_tokens, 2))
        lead_side_2 = list(ngrams(lead1_tokens, 2)) + list(ngrams(lead2_tokens, 2))
        inconst_side_2 = list(ngrams(inconst1_tokens, 2)) + list(ngrams(inconst2_tokens, 2))
        neg_side_2 = list(ngrams(neg1_tokens, 2)) + list(ngrams(neg2_tokens, 2))
        ctr_side_2 = list(ngrams(ctr1_tokens, 2)) + list(ngrams(ctr2_tokens, 2))
        inc_neg_side_2 = list(ngrams(inconst_neg1_tokens, 2)) + list(ngrams(inconst_neg2_tokens, 2))
        '''
    else:
        prec_range_val = 3
        gold_side_1 = gold1_tokens + gold2_tokens + gold3_tokens
        rand_side_1 = rand1_tokens + rand2_tokens + rand3_tokens
        '''
        hybrid_side_1 = hybrid1_tokens + hybrid2_tokens + hybrid3_tokens
        lead_side_1 = lead1_tokens + lead2_tokens + lead3_tokens
        inconst_side_1 = inconst1_tokens + inconst2_tokens + inconst3_tokens
        neg_side_1 = neg1_tokens + neg2_tokens + neg3_tokens
        ctr_side_1 = ctr1_tokens + ctr2_tokens + ctr3_tokens
        inc_neg_side_1 = inconst_neg1_tokens + inconst_neg2_tokens + inconst_neg3_tokens
        '''
        
        gold_side_2 = list(ngrams(gold1_tokens, 2)) + list(ngrams(gold2_tokens, 2)) + list(ngrams(gold3_tokens, 2))
        rand_side_2 = list(ngrams(rand1_tokens, 2)) + list(ngrams(rand2_tokens, 2)) + list(ngrams(rand3_tokens, 2))
        '''
        hybrid_side_2 = list(ngrams(hybrid1_tokens, 2)) + list(ngrams(hybrid2_tokens, 2)) + list(
            ngrams(hybrid3_tokens, 2))
        lead_side_2 = list(ngrams(lead1_tokens, 2)) + list(ngrams(lead2_tokens, 2)) + list(ngrams(lead3_tokens, 2))
        inconst_side_2 = list(ngrams(inconst1_tokens, 2)) + list(ngrams(inconst2_tokens, 2)) + list(
            ngrams(inconst3_tokens, 2))
        neg_side_2 = list(ngrams(neg1_tokens, 2)) + list(ngrams(neg2_tokens, 2)) + list(
            ngrams(neg3_tokens, 2))
        ctr_side_2 = list(ngrams(ctr1_tokens, 2)) + list(ngrams(ctr2_tokens, 2)) + list(
            ngrams(ctr3_tokens, 2))
        inc_neg_side_2 = list(ngrams(inconst_neg1_tokens, 2)) + list(ngrams(inconst_neg2_tokens, 2)) + list(
            ngrams(inconst_neg3_tokens, 2))
        '''


    rouge1_rand_list.append(rouge_one(rand_side_1, gold_side_1))
    '''
    rouge1_lead_list.append(rouge_one(lead_side_1, gold_side_1))
    rouge1_hybrid1_list.append(rouge_one(hybrid_side_1, gold_side_1))
    rouge1_inconst_list.append(rouge_one(inconst_side_1, gold_side_1))
    rouge1_neg_list.append(rouge_one(neg_side_1, gold_side_1))
    rouge1_ctr_list.append(rouge_one(ctr_side_1, gold_side_1))
    rouge1_inconst_neg_list.append(rouge_one(inc_neg_side_1, gold_side_1))
    '''


    rouge2_rand_list.append(rouge_one(rand_side_2, gold_side_2))
    '''
    rouge2_lead_list.append(rouge_one(lead_side_2, gold_side_2))
    rouge2_hybrid1_list.append(rouge_one(hybrid_side_2, gold_side_2))
    rouge2_inconst_list.append(rouge_one(inconst_side_2, gold_side_2))
    rouge2_neg_list.append(rouge_one(neg_side_2, gold_side_2))
    rouge2_ctr_list.append(rouge_one(ctr_side_2, gold_side_2))
    rouge2_inconst_neg_list.append(rouge_one(inc_neg_side_2, gold_side_2))
    '''
    # 1 : X.AUTHID, 2 : spec_domain, 3: site.content  4-6: GOLD, 7-9: LEAD, 10-12: Hybrid, 13-15: Incons, 16-18: Neg, 19-21: Ctr, 22-24: InconsNeg
    rouge_lcs_rand_list.append(r_summ_evaluate.rouge_l(" ".join(summ_data.iloc[row_ind, 3:(3+prec_range_val)]), " ".join(rand_data.iloc[row_ind, 9:(9+prec_range_val)])))
    '''
    rouge_lcs_lead_list.append(
        r_summ_evaluate.rouge_l(" ".join(summ_data.iloc[row_ind, 3:(3+prec_range_val)]), " ".join(rand_data.iloc[row_ind, 9:(9+prec_range_val)])))
    rouge_lcs_hybrid1_list.append(r_summ_evaluate.rouge_l(" ".join(summ_data.iloc[row_ind, 3:(3+prec_range_val)]),
                                                          " ".join(summ_data.iloc[row_ind, 9:(9+prec_range_val)])))
    rouge_lcs_inconst_list.append(r_summ_evaluate.rouge_l(" ".join(summ_data.iloc[row_ind, 3:(3+prec_range_val)]),
                                                          " ".join(summ_data.iloc[row_ind, 12:(12+prec_range_val)])))
    rouge_lcs_neg_list.append(r_summ_evaluate.rouge_l(" ".join(summ_data.iloc[row_ind, 3:(3+prec_range_val)]),
                                                          " ".join(summ_data.iloc[row_ind, 15:(15+prec_range_val)])))
    rouge_lcs_ctr_list.append(r_summ_evaluate.rouge_l(" ".join(summ_data.iloc[row_ind, 3:(3+prec_range_val)]),
                                                          " ".join(summ_data.iloc[row_ind, 18:(18+prec_range_val)])))
    rouge_lcs_inconst_neg_list.append(r_summ_evaluate.rouge_l(" ".join(summ_data.iloc[row_ind, 3:(3+prec_range_val)]),
                                                              " ".join(summ_data.iloc[row_ind, 21:(21+prec_range_val)])))
                                                              
    '''

In [50]:
print('ROUGE-1 scores')
print('Random : ', np.mean(rouge1_rand_list))
print('Hybrid : ', np.mean(rouge1_hybrid1_list))
print('Lead : ', np.mean(rouge1_lead_list))
print('Inconst : ', np.mean(rouge1_inconst_list))
print('Neg : ', np.mean(rouge1_neg_list))
print('Centrality : ', np.mean(rouge1_ctr_list))
print('Inconst Neg : ', np.mean(rouge1_inconst_neg_list))


print('ROUGE-2 scores')
print('Random : ', np.mean(rouge2_rand_list))
print('Hybrid : ', np.mean(rouge2_hybrid1_list))
print('Lead : ', np.mean(rouge2_lead_list))
print('Inconst : ', np.mean(rouge2_inconst_list))
print('Neg : ', np.mean(rouge2_neg_list))
print('Centrality : ', np.mean(rouge2_ctr_list))
print('Inconst Neg : ', np.mean(rouge2_inconst_neg_list))


print("ROUGE-LCS scores")
print('Random : ', np.mean(rouge_lcs_rand_list))
print('Hybrid : ', np.mean(rouge_lcs_hybrid1_list))
print('Lead : ', np.mean(rouge_lcs_lead_list))
print('Inconst : ', np.mean(rouge_lcs_inconst_list))
print('Neg : ', np.mean(rouge_lcs_neg_list))
print('Centrality : ', np.mean(rouge_lcs_ctr_list))
print('Inconst Neg : ', np.mean(rouge_lcs_inconst_neg_list))

ROUGE-1 scores
Random :  0.1379990599629533
Hybrid :  0.5723043386086324
Lead :  0.43833928912911013
Inconst :  0.4962117692705873
Neg :  0.494803718844571
Centrality :  0.5416220637974137
Inconst Neg :  0.5043417976225687
ROUGE-2 scores
Random :  0.008718634895175735
Hybrid :  0.4730777253404862
Lead :  0.3007654481673484
Inconst :  0.3840455721567661
Neg :  0.37341095256276957
Centrality :  0.43613179860418394
Inconst Neg :  0.3932613977905022
ROUGE-LCS scores
Random :  0.7255717860007534
Hybrid :  0.8232203573820384
Lead :  0.770762490973131
Inconst :  0.8054239898789449
Neg :  0.7894715111302092
Centrality :  0.8209413254242048
Inconst Neg :  0.8075944226610319


In [29]:
print(stats.ttest_rel(rouge1_hybrid1_list, rouge1_ctr_list))

Ttest_relResult(statistic=1.3123249837697712, pvalue=0.1979520411983631)

In [30]:
print(stats.ttest_rel(rouge2_hybrid1_list, rouge2_ctr_list))

Ttest_relResult(statistic=1.2599870754072011, pvalue=0.2160107831525605)

In [31]:
print(stats.ttest_rel(rouge_lcs_hybrid1_list, rouge_lcs_ctr_list))

Ttest_relResult(statistic=0.19966515535866647, pvalue=0.8428980002914478)

In [32]:
print(stats.ttest_rel(rouge1_hybrid1_list, rouge1_lead_list))
print(stats.ttest_rel(rouge2_hybrid1_list, rouge2_lead_list))
print(stats.ttest_rel(rouge_lcs_hybrid1_list, rouge_lcs_lead_list))

Ttest_relResult(statistic=3.2304435153740427, pvalue=0.0026900553972566436)
Ttest_relResult(statistic=3.2348940968603572, pvalue=0.002658077730894551)
Ttest_relResult(statistic=2.660039110313475, pvalue=0.011709142494206784)


In [33]:
print(stats.ttest_rel(rouge1_hybrid1_list, rouge1_inconst_list))
print(stats.ttest_rel(rouge2_hybrid1_list, rouge2_inconst_list))
print(stats.ttest_rel(rouge_lcs_hybrid1_list, rouge_lcs_inconst_list))

Ttest_relResult(statistic=2.345806212702816, pvalue=0.02477968165451398)
Ttest_relResult(statistic=2.0434708954681584, pvalue=0.048590384778446294)
Ttest_relResult(statistic=1.2890249140467993, pvalue=0.205844182447398)


In [35]:
print(stats.ttest_rel(rouge1_hybrid1_list, rouge1_inconst_neg_list))
print(stats.ttest_rel(rouge2_hybrid1_list, rouge2_inconst_neg_list))
print(stats.ttest_rel(rouge_lcs_hybrid1_list, rouge_lcs_inconst_neg_list))

Ttest_relResult(statistic=1.9987772893658038, pvalue=0.05344591745686614)
Ttest_relResult(statistic=1.7824144704787328, pvalue=0.08335868116796401)
Ttest_relResult(statistic=1.106782651033, pvalue=0.27593705371796634)


In [51]:
print(stats.ttest_rel(rouge1_hybrid1_list, rouge1_rand_list))
print(stats.ttest_rel(rouge2_hybrid1_list, rouge2_rand_list))
print(stats.ttest_rel(rouge_lcs_hybrid1_list, rouge_lcs_rand_list))

Ttest_relResult(statistic=15.215449773246952, pvalue=5.3312041849194874e-17)
Ttest_relResult(statistic=12.461572666833014, pvalue=1.993936954633146e-14)
Ttest_relResult(statistic=4.420736624137747, pvalue=9.097403440905155e-05)


In [52]:
print(stats.ttest_rel(rouge1_hybrid1_list, rouge1_neg_list))
print(stats.ttest_rel(rouge2_hybrid1_list, rouge2_neg_list))
print(stats.ttest_rel(rouge_lcs_hybrid1_list, rouge_lcs_neg_list))

Ttest_relResult(statistic=2.627527160825993, pvalue=0.012681093198544711)
Ttest_relResult(statistic=2.602878483920757, pvalue=0.013467033146981925)
Ttest_relResult(statistic=3.4325207756338116, pvalue=0.0015527964733779275)
